In [1]:
import os
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import pickle

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
llm.invoke("What's the world's funniest joke? Has there ever been any scientific analysis?")

"\n\nNo, there has not been any scientific analysis to determine the world's funniest joke, as humor is so subjective. However, there have been polls conducted to determine the most popular jokes. According to a 2014 survey by the website Laugh Lab, the following joke was the most popular among British respondents: \n\n“Two hunters are out in the woods when one of them collapses. He doesn’t seem to be breathing and his eyes are glazed. The other guy whips out his phone and calls the emergency services. He gasps: “My friend is dead! What can I do?” The operator says: “Calm down. I can help. First, let’s make sure he’s dead.” There is a silence, then a gunshot is heard. Back on the phone, the guy says: “OK, now what?”"

In [9]:
def tokenizer(text):
    return word_tokenize(text.lower())
queries = {}

idx = None
with open("../data/cisi/CISI.QRY") as f:
    for query in f.read().split(".I"):
        for i, line in enumerate(query.split("\n")):
            if not line:
                continue
            elif i == 0:
                idx = int(line)
                queries[idx] = ""
            elif not line.startswith("."):
                queries[idx] += " "+line

In [ ]:
# queries

In [13]:
answers_openai = {}
for idx, qry in queries.items():
    answers_openai[idx] = llm.invoke(qry)

In [14]:
openai_answers_file_path = './backups/answers_openai.pkl'

with open(openai_answers_file_path, 'wb') as file:
    pickle.dump(answers_openai, file)


print(f"Answers saved to {openai_answers_file_path}")

Answers saved to ./backups/answers_openai.pkl
